In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from re import sub
import time
import random
from faker import Factory
from easymoney.money import EasyPeasy

Use data from the numbers website to get movie title, budget numbers, box office numbers and worldwide numbers

In [2]:
#Get base URL and parse XTML of that URL
gross_base_URL = 'http://www.the-numbers.com/movie/budgets/all'
gross_r = requests.get('http://www.the-numbers.com/movie/budgets/all')
movie_num = BeautifulSoup(gross_r.text,'lxml')

In [3]:
#Get numbers and movie name html childs
movie_num_tr = movie_num.find_all('td',class_='data')
movie_num_names = movie_num.find_all('b')

In [28]:
#Get all movie names
movie_names = ([td.get_text().replace('â\x80\x99',"'") for td in movie_num_names[1:]])

#Encode movies to get the proper text
for mov in range(0,len(movie_names)):
    movie_names[mov] = movie_names[mov].encode('ascii',"ignore")

In [44]:
#Get budget, domestic and worldwide information
budget = []
domestic = []
worldwide = []
rego = 4

for tr in movie_num_tr:
    if rego ==0:
        rego = 4
    rego -=1
    num = int(re.sub("[^0-9|.]","",tr.get_text()))
    if rego ==2:
        budget.append(num)
    if rego ==1:
        domestic.append(num)
    if rego ==0:
        worldwide.append(num)

In [2]:
#Remove those items that aren't movies
del movie_names[len(movie_names)-(len(movie_names) - len(budget)):len(movie_names)]

#Add a movie ID list
movie_id.extend(range(0,len(movie_names)))

Now go to the IMDB website and get the unique urls for each movie

In [100]:
#Get base URL for IMDB search
imdb_base = 'http://www.imdb.com/find?ref_=nv_sr_fn&q={0}&s=all'

In [116]:
#Loop through the movie names to get the unique URLs for each movie
imdb_url = []
movies_dict ={}
count = 0
for movie in movie_names:
    imdb_search = requests.get(imdb_base.format(movie))
    imdb_soup = BeautifulSoup(imdb_search.text,'lxml')
    try:
        if('name' in imdb_soup.find('td', class_='result_text').a['href']) or ('company' in imdb_soup.find('td', class_='result_text')) or ('Video' in imdb_soup.find('td', class_='result_text').text) or ('TV' in imdb_soup.find('td', class_='result_text').text):
            if 'and' in movie.decode():
                imdb_search = requests.get(imdb_base.format(movie.decode().replace('and','')))
                imdb_soup = BeautifulSoup(imdb_search.text,'lxml')
                imdb_url.append(imdb_soup.find('td',class_='result_text').a['href'])
            else:
                imdb_url.append(None)
        else:
            imdb_url.append(imdb_soup.find('td',class_='result_text').a['href'])
    except AttributeError:
        imdb_url.append(None)
    movies_dict[movie]=imdb_url[count]
    count += 1
    time.sleep(random.uniform(.02,5)) #put breaks into the request

In [140]:
#Get base for IMDB website
fake = Factory.create()
header = {'User-Agent':fake.chrome()}
imdb_page_base = 'http://www.imdb.com{0}'
imdb_page = requests.get(imdb_page_base.format(imdb_url[10]),headers = header)
imdb_p_soup = BeautifulSoup(imdb_page.text,'lxml')


In [ ]:
#Loop through each URL to get specific data about each movie
imdb_rating = []
imdb_num_rating=[]
genres = []
director = []
Oscar = []
meta_score = []
mov_year = []
casts = []
mov_count = 0

for imdb in imdb_url:
    if imdb == None:
        imdb_rating.append(None)
        imdb_num_rating.append(None)
        genres.append(None) 
        director.append(None)
        Oscar.append(None)
        meta_score.append(None)
        mov_year.append(None)
        casts.append(None)
        continue
    
    fake = Factory.create()
    header = {'User-Agent':fake.chrome()}
    imdb_page = requests.get(imdb_page_base.format(imdb),headers = header)
    imdb_p_soup = BeautifulSoup(imdb_page.text,'lxml')
    
    try:
        #Get IMDB rating
        imdb_rating.append(float(imdb_p_soup.find('div',class_='ratingValue').get_text().strip()[0:3]))
    except Exception as e:
        imdb_rating.append(None)
    try:
        #Number of IMDB ratings
        imdb_num_rating.append(int(imdb_p_soup.find('span',itemprop='ratingCount').get_text().replace(',',"")))
    except Exception as e:
        imdb_num_rating.append(None)
        
    
    try:
    #Get all genres
        imdb_genres = []
        imdb_genre = imdb_p_soup.find_all('span',itemprop='genre')
        for gen in imdb_genre:
            imdb_genres.append(gen.get_text())
            #print(imdb_genres)
        genres.append(imdb_genres) 
    except Exception as e:
        genres.setdefault(movie_names[mov_count], []).append(None)
        
    try:
    #Get all genres
        cast_members = []
        cast_info = imdb_p_soup.find_all('span',itemprop='actors')
        for cast in cast_info:
            cast_members.append(cast.get_text().strip().replace(',',''))
            #print(imdb_genres)
        casts.append(cast_members)
        movies_dict2[movie_names[mov_count]]=casts[mov_count]
    except Exception as e:
        #casts.setdefault(movie_names[mov_count], []).append(None)
        casts.append(None)
    try:
        #Get director
        director.append(imdb_p_soup.find('span',itemprop='director').get_text().strip())
    except Exception as e:
        director.append(None)
    
    try:    
        #Get oscar
        if ("Won" in imdb_p_soup.find('span',itemprop='awards').get_text().strip()) and ("Oscar" in imdb_p_soup.find('span',itemprop='awards').get_text().strip()):
            Oscar.append('Yes')
        else:
            Oscar.append('No')
    except Exception as e:
        Oscar.append('No')
    
    try:
        #Get the critics scores
        meta_score.append(int(imdb_p_soup.find('div', class_='titleReviewBarSubItem').text.strip()))
    except Exception as e:
        meta_score.append(None)
        
    try:
        #Get movie year
        mov_year.append(int(re.sub(r'([^\s\w]|_)+', '',imdb_p_soup.find('span', id='titleYear').text.split()[0])))
    except Exception as e:
        mov_year.append(None)
        
    mov_count +=1
    time.sleep(random.uniform(.02,.5)) #Put breaks into the request

In [ ]:
movies_dict2

In [ ]:
#Create a dictionary from all of the list
movie_data = {'mov_budget':budget,'mov_domestic':domestic,'mov_worldwide':worldwide,
'mov_imdb_rating':imdb_rating,'mov_imdb_num':imdb_num_rating,'mov_genres':genres,
'mov_director':director,'mov_oscar':Oscar,'meta_score':meta_score, 'mov_name':movie_names,'mov_year':mov_year}

In [ ]:
#Create a dataframe
movie_df = pd.DataFrame(movie_data)

In [ ]:
#Convert the genres list into columns of dummy variables
movie_df['mov_genres'].str.join(sep='*').str.get_dummies(sep='*')
movie_df = pd.concat([movie_df, movie_df['mov_genres'].str.join(sep='*').str.get_dummies(sep='*')], axis=1)


In [ ]:
#Remove movies with a budget of 0 and those that are older than 1960 because the inflation adjustment only goes back to 1960
movie_df = movie_df[(movie_df['mov_domestic']!=0)]
movie_df = movie_df[(movie_df['mov_year']>1960)] # will need to cut out movies older than 1960


In [ ]:
infl_domestic = []
infl_worldwide = []
infl_budget = []
for row in range(0,4980):
    try:
        movie_df.loc[row,'infl_domestic'] = ep.normalize(amount=int(movie_df['mov_domestic'][row]), region="US", from_year=int(join_movie_df['mov_year'][row]), to_year="latest", base_currency="USD")
        movie_df.loc[row,'infl_worldwide'] = ep.normalize(amount=int(movie_df['mov_worldwide'][row]), region="US", from_year=int(join_movie_df['mov_year'][row]), to_year="latest", base_currency="USD")
        movie_df.loc[row,'infl_budget'] = ep.normalize(amount=int(movie_df['mov_budget'][row]), region="US", from_year=int(join_movie_df['mov_year'][row]), to_year="latest", base_currency="USD")
    except Exception as e:
        pass

In [ ]:
#Remove any movies with NAs in these variables
movie_df = movie_df[np.isfinite(movie_df['mov_imdb_rating'])]
movie_df = movie_df[np.isfinite(movie_df['mov_year'])]

In [ ]:
#Make dataframe into csv
movie_df.to_csv('join_movie_df.csv')

In [ ]:
#Read movie and remove encoding
movie_df = pd.read_csv('movie_df.csv',encoding = "ISO-8859-1")